In [ ]:
from collections import OrderedDict as odict
from pathlib import Path
import geopandas as gpd
import pandas as pd

import colorcet
import param
import panel
import datashader as ds
import intake
import dask.dataframe

import hvplot.pandas

import holoviews
from holoviews.element import tiles
from holoviews.operation.datashader import rasterize, shade, spread
# from holoviews.element.tiles import EsriStreet

import seaborn
import datashader.transfer_functions as tf

import cartopy.crs as ccrs

# This will become default in pandas 3.0
pd.options.mode.copy_on_write = True

holoviews.extension('bokeh', logo=False)

In [ ]:
class Explorer(param.Parameterized):
    cmap_list = ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']
    cmaps = odict([(n, colorcet.palette[n]) for n in cmap_list])
    cmap = param.Selector(cmaps)

    maps = ['OSM', 'CartoDark', 'CartoLight', 'OpenTopoMap']
    bases = odict([(name, getattr(tiles, name)().relabel(name)) for name in maps])
    basemap = param.Selector(bases)
    
    spreading = param.Integer(0, bounds=(0, 5))
    data_opacity = param.Magnitude(1.00)
    map_opacity = param.Magnitude(0.75)
    show_labels = param.Boolean(False)
    
    norms  = odict(Histogram_Equalization='eq_hist', Linear='linear', Log='log', Cube_root='cbrt')
    normalisation = param.Selector(norms)
    
    gopts = holoviews.opts.Tiles(responsive=True, xaxis='bottom', yaxis='left', show_grid=True)

    catalog = intake.open_catalog(Path('catalog.yml'))
    datasets = list(catalog.keys())
    dataset = param.Selector(datasets)

    @param.depends('dataset')
    def get_polygons(self):
        source = getattr(self.catalog, self.dataset)
        gdf = source.read()[source.metadata['usecols']]
        
        # WEB MERCATOR (EPSG: 3857) to match basemaps
        gdf = gdf.to_crs("epsg:3857")
        target = source.metadata['categories']['general']
        gdf['target_categories'] = gdf[target].astype('category').cat.codes

        return holoviews.Polygons(gdf, vdims=['target_categories', 'geometry'])
        
    @param.depends('map_opacity', 'basemap')
    def set_tiles(self):
        return self.basemap.opts(self.gopts).opts(alpha=self.map_opacity)

    @param.depends('show_labels')
    def set_labels(self):
        return tiles.StamenLabels().options(level='annotation', alpha=1 if self.show_labels else 0)
        
    def viewable(self,**kwargs):
        polygons = holoviews.DynamicMap(self.get_polygons)
        rasterized = rasterize(polygons, aggregator='count_cat')
        shaded = shade(rasterized, color_key=self.param.cmap, cnorm=self.param.normalisation)
        spreaded = spread(shaded, px=self.param.spreading, how="add")
        dataplot = spreaded.apply.opts(alpha=self.param.data_opacity, show_legend=True)

        return holoviews.DynamicMap(self.set_tiles) * holoviews.DynamicMap(self.set_labels) * dataplot

In [ ]:
%%time
explorer = Explorer(name="")

dash = panel.Row(panel.Column(panel.Param(explorer.param, expand_button=False)), explorer.viewable())
dash.servable("Datashader Dashboard")

In [ ]:
catalog = intake.open_catalog(Path('catalog.yml'))
source = getattr(catalog, 'sample_gb_forestry')
gdf = source.read()[source.metadata['usecols']]
gdf = gdf.to_crs("epsg:3857")

In [ ]:
%%time
# https://hvplot.holoviz.org/user_guide/Customization.html
gdf.hvplot(
    # aggregator=ds.count_cat(source.metadata['categories']['general']),
    groupby=source.metadata['categories']['general'], 
    datashade=False, rasterize=False, height=600, width=800, tiles='OSM'#, cmap='hot'
)

In [ ]:
# dash = panel.Row(hv)
# dash.servable("Datashader Dashboard")